In [2]:
import pandas as pd
import numpy as np
import itertools

In [5]:
bakery1000 = pd.read_csv('../apriori/1000/1000-out1.csv',sep = '\n', header = None)
bakery1000Mtemp = bakery1000.iloc[:,0].values
bakery1000M = [list(map(int, x.split(','))) for x in bakery1000Mtemp]
for x in bakery1000M:
    del x[0]

In [6]:
exampleData = pd.read_csv('../example/out1.csv', sep = '\n', header = None)
exampleDataTemp = exampleData.iloc[:,0].values
exampleDataM = [list(map(int, x.split(','))) for x in exampleDataTemp]
for x in exampleDataM:
    del x[0]

In [85]:
exampleDataM[0:10]

[[2, 5, 27, 29, 32, 48],
 [15, 27, 31, 33, 36],
 [9, 16, 22],
 [12, 14, 16],
 [2, 9, 44],
 [9, 26, 39],
 [10, 31],
 [1, 47, 49],
 [12, 14, 16],
 [12, 14, 16]]

In [8]:
def support(item, baskets):
    top = sum([1 if item in basket else 0 for basket in baskets])
    bottom = len(baskets)
    return top*1.0/bottom

In [9]:
def getPairs(F, k):
    outList = []
    for first in F:
        for second in F:
            if len(first) == len(second) == k:
                outList.append([first, second, tuple(set(first) | set(second))])
    return outList 

In [10]:
def getSubsets(tmpTuple):
    size = len(tmpTuple) - 1
    return set(itertools.combinations(list(tmpTuple), size))

In [11]:
def candidate(F,k):
    C = set()
    for pair in getPairs(F,k):
        if len(pair[2]) == len(pair[0]) + 1:
            c = pair[2]
            flag = True
            for s in getSubsets(c):
                if s not in F:
                    flag = False
            if flag == True:
                if c not in C:
                    C.add(tuple(c))
    return C

In [12]:
def apriori(T, minSup):
    FullSupport = []
    I =  list(set(x for l in T for x in l))
    Fk = []; Fk.append(None);
    firstIter = set(tuple([x]) for x in filter(lambda a: a != -1, [x if support(x, T) >= minSup else -1 for x in I]))
    Fk.append(firstIter)
    k = 2
    while len(Fk[k-1]) != 0:
        Ck = candidate(Fk[k-1], k-1)
        count = {}
        for c in Ck:
            count[c] = 0
        for t in T:
            for c in Ck:
                if set(c) <= set(tuple(t)):
                    count[c] += 1
        tmpFk = []
        for c in Ck:
            #print(count[c]*1.0/len(T))
            if count[c]*1.0/len(T) >= minSup:
                #print(c)
                tmpFk.append(c)
        Fk.append(set(tmpFk))
        newSet = set(x for x in Fk[k-1])
        for newTerm in Fk[k]:
            for oldTerm in Fk[k-1]:
                if set(oldTerm).issubset(newTerm) == True:
                    if oldTerm in newSet:
                        #print("before: ", newSet)
                        newSet.remove(oldTerm)
                        #print("After: ", newSet)
        Fk[k-1] = set(tuple(sorted(x)) for x in newSet)
        #Fk[k] = set(tuple(sorted(x)) for x in Fk[k])
        k += 1
    Fk = Fk[1:-1]
    return Fk
                    
            
        
    
    

In [140]:
def getCorrect():
    t = np.linspace(.008, .03, 75)
    for minS in t:
        ap = apriori(bakery1000M, minS)
        print("---- Apriori: ", minS, " ----")
        print("NumRules: ", len(genRules(bakery1000M, ap, .8)))
        for i, val in enumerate(ap):
            print(i+1, ": ", len(val))
            

In [141]:
getCorrect()

---- Apriori:  0.008  ----
NumRules:  48
1 :  7
2 :  53
3 :  18
4 :  1
---- Apriori:  0.0082972972973  ----
NumRules:  48
1 :  11
2 :  27
3 :  18
4 :  1
---- Apriori:  0.00859459459459  ----
NumRules:  48
1 :  11
2 :  27
3 :  18
4 :  1
---- Apriori:  0.00889189189189  ----
NumRules:  48
1 :  11
2 :  27
3 :  18
4 :  1
---- Apriori:  0.00918918918919  ----
NumRules:  48
1 :  11
2 :  17
3 :  18
4 :  1
---- Apriori:  0.00948648648649  ----
NumRules:  48
1 :  11
2 :  17
3 :  18
4 :  1
---- Apriori:  0.00978378378378  ----
NumRules:  48
1 :  11
2 :  17
3 :  18
4 :  1
---- Apriori:  0.0100810810811  ----
NumRules:  48
1 :  12
2 :  12
3 :  18
4 :  1
---- Apriori:  0.0103783783784  ----
NumRules:  48
1 :  12
2 :  12
3 :  18
4 :  1
---- Apriori:  0.0106756756757  ----
NumRules:  48
1 :  12
2 :  12
3 :  18
4 :  1
---- Apriori:  0.010972972973  ----
NumRules:  48
1 :  12
2 :  12
3 :  18
4 :  1
---- Apriori:  0.0112702702703  ----
NumRules:  48
1 :  12
2 :  11
3 :  18
4 :  1
---- Apriori:  0.011567

In [121]:
def genRules(data, freqItemSet, minConf):
    allRules = []
    freqItemSet = freqItemSet[1:]
    freqItemSet = [item for sublist in freqItemSet for item in sublist]
    for itemSet in freqItemSet:
        for item in itemSet:
            noItem=set(itemSet)-(set([item]))
            item = set([item])
            conf = confidence(data,noItem,item)
            confItem = [noItem, item, conf]
            if conf >= minConf:
                allRules.append(confItem)
    return allRules
                    

In [82]:
def confidence(T,set1,set2): #Dataset,leftSideList,rightSideList
    top = sum([1 if (set1|set2) <= set(basket) else 0 for basket in T])
    bottom = sum([1 if (set1) <= set(basket) else 0 for basket in T])
    if(bottom == 0 | top == 0):
        return 0
    return top*1.0/bottom

In [99]:
t = apriori(exampleDataM, .1)

In [96]:
t

[set(), {(1, 49), (9, 22), (15, 36)}, {(12, 14, 16)}]

In [101]:
t[1:]

[{(1, 49), (9, 22), (15, 36)}, {(12, 14, 16)}]

In [133]:
t = apriori(exampleDataM, .05)

In [134]:
t

[{(41,)}, {(1, 49), (9, 22), (15, 36)}, {(12, 14, 16)}]

In [135]:
genRules(exampleDataM, t, .7)

[[{36}, {15}, 0.7988505747126436],
 [{15}, {36}, 0.7513513513513513],
 [{22}, {9}, 0.8418604651162791],
 [{9}, {22}, 0.8044444444444444],
 [{49}, {1}, 0.7839506172839507],
 [{1}, {49}, 0.8141025641025641],
 [{14, 16}, {12}, 0.9961240310077519],
 [{12, 16}, {14}, 0.9922779922779923],
 [{12, 14}, {16}, 0.9589552238805971]]

In [87]:
type(t[1])

numpy.float64

In [88]:
t[1]

0.001292929292929293